In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions, ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
# from selenium.webdriver.support.expected_conditions import staleness_of
import pandas as pd 
import time
import re
import os
import requests

In [14]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(r"D:\GS_WebScarping\project_7_g3\chromedriver.exe")
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    
    
    # get the username tab
    try:
        provided_u_name = "Mohsin7865"
        user_name = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH,'.//input[@name="ctl00$ContentPlaceHolder1$txtUsername"]')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")
    
    # get password tab
    try:
        provided_pass = "Muhssan7865@"
        password = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/input[@name="ctl00$ContentPlaceHolder1$txtPassword"]')))
        driver.execute_script("arguments[0].scrollIntoView();", password)
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}") 
    
    # get the check done
    try:
        check = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/input[@name="ctl00$ContentPlaceHolder1$btnLogin"]')))
        if check:
            driver.execute_script("arguments[0].scrollIntoView();", check)
            check.click()
        
    except Exception as e:
        print(f"No check tab found and the error is {e}")
    # ======================================================
    
    # total cars --> done
    try:
        cars = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/span[@id ="ContentPlaceHolder1_ContentSearch_ucSearchResults_lblResults"]'))).text.strip()
        if cars:
            match = re.search(r"(\d+)", cars)

            if match:
                total_cars= int(match.group(1))  # Convert to integer
                print(f"{total_cars} number of cars found")
        else:
            print("No cars found")
    except Exception as e:
        print(f"No cars found and error is {e}")
        
    
    # list to append the values
    results = []
    
    count = 0
    
    # while loop 
    while count < total_cars:
        # find per page cars
        try:
            per_page_cars = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/td[@style="width:75px"]')))
            
            # loop around the number of cars
            for i in range(len(per_page_cars)): # total_cars
                if count >= total_cars:
                    break
                # get the page cars and click on each of them
                try:

                    driver.execute_script("arguments[0].scrollIntoView();", per_page_cars[i])
                    time.sleep(1)
                    per_page_cars[i].click()

                    
                    # dict to store the values
                    details = {}
                    
                    # title --> done
                    try:
                        title = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_lblTitle"]'))).text.strip()
                        if title:
                            details['Title'] = title
                        else:
                            details['Title'] = 'na'
                    except Exception as e:
                        print("No car title")
                        
                    # veh all info
                    try:
                        veh_btn_main =WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/div[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_header"]'))) 
                        
                        # veh information button
                        veh_button = WebDriverWait(veh_btn_main, 2).until(EC.presence_of_element_located((By.XPATH, './/span[text()="Vehicle"]')))
                        if veh_button:
                            # time.sleep()
                            veh_button.click()
                            
                            veh_main_card =WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/div[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle"]')))
                            
                            # imgs --> done
                            try:
                                img_card_1 = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/table[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_rptImageList_rptImages_0"]')))
                                img_card_2 = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/table[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_rptImageList_rptImages_1"]')))
                                img_card_3 = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/table[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_rptImageList_rptImages_1"]')))
                                if img_card_1 or img_card_2 or img_card_3:
                                    
                                    img_trs_1 = WebDriverWait(img_card_1, 2).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
                                    img_trs_2 = WebDriverWait(img_card_2, 2).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
                                    img_trs_3 = WebDriverWait(img_card_3, 2).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
                                    imgs_lst = []
                                    for img_tr_1 in img_trs_1:
                                        imgs_1 = img_tr_1.find_elements(By.TAG_NAME, 'a')
                                        for img_1 in imgs_1:
                                            imgs_lst.append(img_1.get_attribute("href"))
                                        
                                    for img_tr_2 in img_trs_2:
                                        imgs_2 = img_tr_2.find_elements(By.TAG_NAME, 'a')
                                        for img_2 in imgs_2:
                                            imgs_lst.append(img_2.get_attribute("href"))
                                            
                                    for img_tr_3 in img_trs_3:
                                        imgs_3 = img_tr_3.find_elements(By.TAG_NAME, 'a')
                                        for img_3 in imgs_3:
                                            imgs_lst.append(img_3.get_attribute("href"))
                                    
                                    imgs_str = ", ".join(imgs_lst)
                                    details['Images'] = imgs_str
                                else:
                                    print("No images from the first card")
                                    
                            except Exception as e:
                                print(f"Error processing images")
                    

                            # veh dets --> done
                            try:
                                model = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblModel"]'))).text.strip()
                                if model:
                                    details['Model'] = model
                                else:
                                    details['Model'] = 'na'
                                    
                                var = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblVariant"]'))).text.strip()
                                if var:
                                    details['Variant'] = var
                                else:
                                    details['Variant'] = 'na'
                                    
                                reg_no = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblRegNo"]'))).text.strip()
                                if reg_no:
                                    details['reg_no'] =reg_no
                                else:
                                    details['reg_no'] = 'na'
                                
                                reg_date= WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblRegDate"]'))).text.strip()
                                if reg_date:
                                    details['reg_date'] = reg_date
                                else:
                                    details['reg_date'] = 'na'
                                
                                mileage = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblMileage"]'))).text.strip()
                                if mileage:
                                    details['mileage'] = mileage
                                else:
                                    details['mileage'] ='na'
                                    
                                color = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblColour"]'))).text.strip()
                                if color:
                                    details['color'] = color
                                else:
                                    details['color'] = 'na'
                                    
                                trans = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblTransmissionType"]'))).text.strip()
                                if trans:
                                    details['Transmission_type'] = trans
                                else:
                                    details['Transmission_type'] = 'na'
                                    
                                fuel = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblFuelType"]'))).text.strip()
                                if fuel:
                                    details['fuel'] = fuel
                                else:
                                    details['fuel'] = 'na'
                                    
                                co2 = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblCO2"]'))).text.strip()
                                if co2:
                                    details['CO2'] = co2
                                else:
                                    details['CO2'] = 'na'
                                    
                                doors = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblDoors"]'))).text.strip()
                                if doors:
                                    details['doors'] = doors
                                else:
                                    details['doors']= 'na'
                                    
                                start = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblAuctionStartDate"]'))).text.strip()
                                if start:
                                    details['Start_time'] = start
                                else:
                                    details['Start_time'] = 'na'
                                    
                                ending = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblAuctionEndDate"]'))).text.strip()
                                if ending:
                                    details['Ending_time'] = ending
                                else:
                                    details['Ending_time']= 'na'
                                    
                                trade_no = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblTradeNo"]'))).text.strip()
                                if trade_no:
                                    details['Trade_no'] = trade_no
                                else:
                                    details['Trade_no'] = trade_no
                                    
                                lot_no = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblLotNumber"]'))).text.strip()
                                if lot_no:
                                    details['Lot_no'] = lot_no
                                else:
                                    details['Lot_no'] = 'na'
                                    
                                curr_bid= WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblCurrentBid"]'))).text.strip()
                                if curr_bid:
                                    details['Current_bid'] = curr_bid
                                else:
                                    details['Current_bid'] = 'na'
                                    
                                auc_fee = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblAuctionFee"]'))).text.strip()
                                if auc_fee:
                                    details['Auction_fee'] = auc_fee
                                else:
                                    details['Auction_fee']='na'
                                    
                                no_bids = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblNoBids"]'))).text.strip()
                                if no_bids:
                                    details['No_of_bids'] = no_bids
                                else:
                                    details['No_of_bids'] = 'na'
                                    
                                no_watchs = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblNoWatchers"]'))).text.strip()
                                if no_watchs:
                                    details['No_of_watchers'] = no_watchs
                                else:
                                    details['No_of_watchers'] = 'na'
                                    
                                close = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblModel"]'))).text.strip()
                                if close:
                                    details['Closes'] = close
                                else:
                                    details['Closes'] = 'na'
                                    
                                buy_price = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblBuyItNow"]'))).text.strip()
                                if buy_price:
                                    details['Buy_It_Now_Price'] = buy_price
                                else:
                                    details['Buy_It_Now_Price']= 'na'
                                    
                                cap_clean = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_ucGuidePricing_lblCAPConditionValuation"]'))).text.strip()
                                if cap_clean:
                                    details['Cap_clean'] = cap_clean
                                else:
                                    details['Cap_clean'] = 'na'
                                    
                                cap_ret = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_ucGuidePricing_lblCAPRetailValuation"]'))).text.strip()
                                if cap_ret:
                                    details['Cap_retail'] = cap_ret
                                else:
                                    details['Cap_retail'] = 'na'
                                
                                glasses = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_ucGuidePricing_lblGlasses"]'))).text.strip()
                                if glasses:
                                    details['glasses'] = glasses
                                else:
                                    details['glasses'] = 'na'
                                
                                del_charges = WebDriverWait(veh_main_card, 2).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabVehicle_lblDeliveryCharge"]'))).text.strip()
                                if del_charges:
                                    details['Delivery_charges'] = del_charges
                                else:
                                    details['Delivery_charges'] = 'na'         
                            except Exception as e:
                                print(f"No veh det found")        
                                
                        else:
                            print("no veh info button")                                        
                    
                        # Additional Info --> done
                        add_info_button = WebDriverWait(veh_btn_main, 2).until(EC.presence_of_element_located((By.XPATH, './/span[text()="Additional Info"]')))
                        if add_info_button:
                            time.sleep(2)
                            add_info_button.click()
                            
                            try:
                                add_main = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/div[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabAdditionalInfo"]')))
                                if add_main:
                                    add_rows = add_main.find_elements(By.TAG_NAME, 'tr')
                                    for add_row in add_rows:
                                        add_tds = add_row.find_elements(By.TAG_NAME, 'td')
                                        
                                        if len(add_tds) == 2:
                                            add_lbl = add_tds[0]
                                            add_val  =add_tds[1]
                                            details[add_lbl.text.strip()] = add_val.text.strip()

                                    serv_hist =WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/table[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_tabAdditionalInfo_gdvServiceHistory"]'))) 
                                    if serv_hist:
                                        serv_rows = serv_hist.find_elements(By.TAG_NAME, 'tr')[1:]
                                        type_values = [row.find_elements(By.TAG_NAME, 'td')[0].text for row in serv_rows]
                                        details['Type'] = type_values
                                        date_values = [row.find_elements(By.TAG_NAME, 'td')[1].text for row in serv_rows]
                                        details['Date']  =date_values
                                        miles_values = [row.find_elements(By.TAG_NAME, 'td')[2].text for row in serv_rows]
                                        details['Miles'] = miles_values
                                        dealership_values = [row.find_elements(By.TAG_NAME, 'td')[3].text for row in serv_rows]
                                        details['Dealership'] = dealership_values                 
                                else:
                                    print("No add info found")
                                    
                            except Exception as e:
                                print("No add info found")
                        else:
                            print("No add info button")
                            
                        
                        
                        # Specification --> done
                        spec_info_button = WebDriverWait(veh_btn_main, 2).until(EC.presence_of_element_located((By.XPATH, './/span[text()="Specification"]')))
                        if spec_info_button:
                            time.sleep(2)
                            spec_info_button.click()
                            

                            # standard specs --> done
                            try:
                                stnd_spec_card = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/div[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_TabPanel1_divSpec"]')))
                                if stnd_spec_card:
                                    std_uls = WebDriverWait(stnd_spec_card, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/ul[starts-with(@style, "list-style-image: url(images/bullet.gif)")]')))
                                    std_val_lst = []
                                    for std_ul in std_uls:
                                        std_vals = WebDriverWait(std_ul, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/li[starts-with(@style, "margin-left:13px")]')))
                                        # std_val_lst = [std_val.text.strip() for std_val in std_vals]
                                        for std_val in std_vals:
                                            std_val_lst.append(std_val.text.strip())
                                    details['Standard_specs'] = std_val_lst
                                else:
                                    print("No standard specs")
                            except Exception as e:
                                print(f"No stnadard specs")
                            
                            # additional specs --> check
                            try:
                                add_hist_spec_card = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_TabPanel1_divDFA"]')))
                                if add_hist_spec_card:
                                    add_hist_uls = WebDriverWait(add_hist_spec_card, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/ul[starts-with(@style, "list-style-image: url(images/bullet.gif)")]')))
                                    add_hist_val_lst = []
                                    for add_hist_ul in add_hist_uls:
                                        add_hist_vals = WebDriverWait(add_hist_ul, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/li[starts-with(@style, "margin-left:13px")]')))
                                        # std_val_lst = [std_val.text.strip() for std_val in std_vals]
                                        for add_hist_val in add_hist_vals:
                                            add_hist_val_lst.append(add_hist_val.text.strip())
                                    details['Additional_specs'] = add_hist_val_lst
                                else:
                                    print("No add specs")
                            except Exception as e:
                                print(f"No add specs")
                        else:
                            print("No specs button to clk")
                        
                        
                        # Condition --> done
                        cond_info_button = WebDriverWait(veh_btn_main, 3).until(EC.presence_of_element_located((By.XPATH, './/span[text()="Condition"]')))
                        if cond_info_button:
                            time.sleep(2)
                            cond_info_button.click()
                            
                            
                            try:
                                cond_det_card = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_TabPanel2"]')))
                                if cond_det_card:
                                    
                                    # get img --> done
                                    try:
                                        cond_img = WebDriverWait(cond_det_card, 3).until(EC.presence_of_element_located((By.XPATH, './/img[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_TabPanel2_imgCarSplat"]'))).get_attribute("src")
                                        if cond_img:
                                            details['Cond_image'] = cond_img
                                        else:
                                            details['Cond_image'] = 'na'
                                    except Exception as e:
                                        print(f"No cond image")
                                        
                                    # get other info --> done
                                    try:
                                        cond_info_upper = WebDriverWait(cond_det_card, 3).until(EC.presence_of_element_located((By.XPATH, './/td[@style="width: 325px; vertical-align:top;"]')))
                                        if cond_info_upper:
                                            cond_upper_trs = cond_info_upper.find_elements(By.TAG_NAME, 'tr')
                                            for cond_tr in cond_upper_trs:
                                                cond_up_lbl = cond_tr.find_elements(By.TAG_NAME, 'td')[0].text.strip()
                                                cond_up_val = cond_tr.find_elements(By.TAG_NAME, 'td')[1].text.strip()
                                                details[cond_up_lbl] = cond_up_val
                                            
                                        else:
                                            print("No cond upper vals")    
                                        cond_info_mid = WebDriverWait(cond_det_card, 3).until(EC.presence_of_element_located((By.XPATH, './/td[@style="width: 325px; vertical-align:top"]')))
                                        if cond_info_mid:
                                            cond_mid_trs = cond_info_mid.find_elements(By.TAG_NAME, 'tr')
                                            for cond_mid_tr in cond_mid_trs:
                                                cond_mid_lbl = cond_mid_tr.find_elements(By.TAG_NAME, 'td')[0].text.strip()
                                                cond_mid_val = cond_mid_tr.find_elements(By.TAG_NAME, 'td')[1].text.strip()
                                                details[cond_mid_lbl] = cond_mid_val
                                        else:
                                            print("No middle information")
                                        
                                        cond_note =WebDriverWait(cond_det_card, 3).until(EC.presence_of_element_located((By.XPATH, './/span[@id="ContentPlaceHolder1_ContentSearch_tbcSelling_TabPanel2_lblConditionNotes"]'))).text.strip()
                                        if cond_note:
                                            details['Condition_Notes'] = cond_note
                                        else:
                                            details['Condition_Notes'] = "na"
                                    except Exception as e:
                                        print(f"No upper vals")
                                else:
                                    print("No cond main card")
                            except Exception as e:
                                print(f"No cond main card")        
                            
                    except Exception as e:
                        print(f"No veh button")       
                    count+=1  
                    results.append(details)
                    driver.back()
                except Exception as e:
                    print("No cars found")
                              
                    
            if count % 10 ==0:
                try:
                    # main_card_next = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class = "justify-content-center row"]')))
                    # next_rol_card =WebDriverWait(main_card_next, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@role="group"]'))) 
                    next_link = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(//a[text() = "Next"])[2]')))
                    if next_link:
                        driver.execute_script("arguments[0].scrollIntoView();", next_link)
                        next_link.click()
                        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/td[@style="width:75px"]')))
                        
                except Exception as e:
                    print("No next found")
                    break
        except Exception as e:
            print("No cars found")
            break
        
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("agnew.csv", index=False)
    # time.sleep(5)
    driver.quit()
path = "https://www.agnewtradecentre.com/vehicles.aspx"
scrape(path)

27 number of cars found
No veh det found
No add specs
No add specs
No add specs
No veh det found
No add specs
No add specs
No veh det found
No add specs
No add specs
No veh det found
No add specs
No veh det found
No add specs
No add specs
No add specs
No veh det found
No add specs
No add specs
No add specs
No add specs
No veh det found
No add specs
No veh det found
No add specs
No veh det found
No add specs
No add specs
No veh det found
No add specs
No veh det found
No add specs
No add specs
No add specs
No veh det found
No add specs
No add specs
No add specs
No veh det found
No add specs


In [15]:
df=pd.read_csv("agnew.csv")
df

,Title,Images,Model,Variant,reg_no,reg_date,mileage,color,Transmission_type,fuel,...,OSR:,Spare:,Condition_Notes,No_of_watchers,Closes,Buy_It_Now_Price,Cap_clean,Cap_retail,glasses,Delivery_charges
0,Skoda Octavia Estate,https://www.agnewtradecentre.com/images/vehicl...,Octavia Estate,1.0 Tsi Se Technology 5dr,SRZ2263,27/01/2021,84452,WHITE,Manual,Petrol,...,2.0 mm,No,One owner 2021 Octavia Estate Se Technology wi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Seat Leon Diesel Hatchback,https://www.agnewtradecentre.com/images/vehicl...,Leon Diesel Hatchback,2.0 Tdi Fr 5dr [Technology Pack],KU66VYR,07/09/2016,145008,BLUE,Manual,Diesel,...,2.0 mm,No,"2016 Leon Fr with technology pack, good histor...",1.0,Leon Diesel Hatchback,"£3,485","£3,263","£5,063",NaN,£349
2,Audi Q3 Estate,https://www.agnewtradecentre.com/images/vehicl...,Q3 Estate,1.4t Fsi Sport 5dr S Tronic,TIG1650,07/12/2017,57476,GLACIER WHITE,Semi Automatic,na,...,2.0 mm,Yes,2017 Q3 Sport with Audi history & MOT until No...,3.0,Q3 Estate,"£10,960","£10,903","£13,288",NaN,£349
3,Mazda Mazda3 Hatchback,https://www.agnewtradecentre.com/images/vehicl...,Mazda3 Hatchback,2.0 Se-L Nav 5dr,JRZ3032,30/04/2015,106622,SILVER,Manual,Petrol,...,3.0 mm,No,2015 Mazda 3 Se-L with nav and heated front se...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mercedes-Benz C Class Diesel Coupe,https://www.agnewtradecentre.com/images/vehicl...,C Class Diesel Coupe,C250d 4matic Amg Line Premium 2dr Auto,DYZ2783,24/02/2017,100713,SILVER,Automatic,na,...,3.0 mm,No,2017 Mercedes C250d 4 matic Amg Line Premium w...,1.0,C Class Diesel Coupe,"£11,248","£10,600","£13,045",NaN,£349
5,Volkswagen Golf Diesel Hatchback,https://www.agnewtradecentre.com/images/vehicl...,Golf Diesel Hatchback,2.0 Tdi Gtd 5dr [Nav],ORZ8475,06/03/2017,139788,GREY,Manual,Diesel,...,2.0 mm,Yes,2017 Golf Gtd with nav (No SD card)and heated ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Audi A3 Diesel Saloon,https://www.agnewtradecentre.com/images/vehicl...,A3 Diesel Saloon,2.0 Tdi 184 Quattro S Line 4dr S Tronic [Nav],DG16CZY,02/03/2016,122242,BLACK,Automatic,Diesel,...,3.0 mm,Yes,"2016 A3 Quattro S Line Saloon with nav, 1 key ...",4.0,A3 Diesel Saloon,"£6,900","£6,948","£8,919",NaN,£349
7,Mazda Cx-5 Diesel Estate,https://www.agnewtradecentre.com/images/vehicl...,Cx-5 Diesel Estate,2.2d [175] Sport Nav 5dr Awd,LXZ5997,23/02/2018,74722,RED,Manual,Diesel,...,2.0 mm,No,"2018 Cx-5 Sport Nav, full service history, sat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Bmw 5 Series Diesel Saloon,https://www.agnewtradecentre.com/images/vehicl...,5 Series Diesel Saloon,520d Se 4dr,YK60HRW,21/09/2010,127224,BLUE,Manual,Diesel,...,3.0 mm,No,"2010 520d Se, condition to be expected for its...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Skoda Kodiaq Diesel Estate,https://www.agnewtradecentre.com/images/vehicl...,Kodiaq Diesel Estate,2.0 Tdi 190 Edition 4x4 5dr Dsg [7 Seat],MRZ4845,31/05/2017,88822,RED,Automatic,Diesel,...,2.0 mm,Yes,"2017 Kodiaq Edition 4x4 Dsg with leather, heat...",4.0,Kodiaq Diesel Estate,"£13,160","£13,430","£15,566",NaN,£349


### images download

In [16]:
reg_img = df[['reg_no', 'Images']]

In [18]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

def download_images(data, main_folder="Images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["reg_no"] # separate reg nums
        image_urls = row["Images"].split(", ")  # Split image URLs by comma if multiple
        
        # Create a subfolder for the car registration number
        reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
        os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
        
        # loop around the urls of the current row and also save index for further use
        for idx, url in enumerate(image_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                url = urljoin("https://", url) # sets the urls starting from https://.....
            
            # parse the url
            parsed_url = urlparse(url)

            # check if the parsed url is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                continue
            
            # try downloading images
            try:
                # Download the image
                response = requests.get(url, stream=True) # send the url for downloading
                response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
                # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                # If no file name is found, assigns a default name based on the index.
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                # Extracts the file extension (e.g., jpg). from the last index value
                file_extension = file_name.split(".")[-1]
                
                # Ensure the file has a valid extension
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                
                # Construct the full path for the file
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")
# Call the function
download_images(reg_img)


Downloaded: Images\SRZ2263\SRZ2263_1_image_1.jpg
Downloaded: Images\SRZ2263\SRZ2263_2_image_2.jpg
Downloaded: Images\SRZ2263\SRZ2263_3_image_3.jpg
Downloaded: Images\SRZ2263\SRZ2263_4_image_4.jpg
Downloaded: Images\SRZ2263\SRZ2263_5_image_5.jpg
Downloaded: Images\SRZ2263\SRZ2263_6_image_6.jpg
Downloaded: Images\SRZ2263\SRZ2263_7_image_7.jpg
Downloaded: Images\SRZ2263\SRZ2263_8_image_8.jpg
Downloaded: Images\SRZ2263\SRZ2263_9_image_9.jpg
Downloaded: Images\SRZ2263\SRZ2263_10_image_10.jpg
Downloaded: Images\SRZ2263\SRZ2263_11_image_11.jpg
Downloaded: Images\SRZ2263\SRZ2263_12_image_12.jpg
Downloaded: Images\SRZ2263\SRZ2263_13_image_13.jpg
Downloaded: Images\SRZ2263\SRZ2263_14_image_14.jpg
Downloaded: Images\SRZ2263\SRZ2263_15_image_15.jpg
Downloaded: Images\SRZ2263\SRZ2263_16_image_16.jpg
Downloaded: Images\SRZ2263\SRZ2263_17_image_17.jpg
Downloaded: Images\SRZ2263\SRZ2263_18_image_18.jpg
Downloaded: Images\SRZ2263\SRZ2263_19_image_19.jpg
Downloaded: Images\SRZ2263\SRZ2263_20_image_20.jp